In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import KFold
import xgboost as xgb

/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/usr/local/lib/python3.5/dist-packages/xgboost/__init__.py:29: FutureWarning: Python 3.5 support is deprecated; XGBoost will require Python 3.6+ in the near future. Consider upgrading to Python 3.6+.
  FutureWarning)


In [2]:
train_v3 = pd.read_csv('dataset_v3')

In [3]:
train_labels = train_v3['damage_grade']
del train_v3['damage_grade']


In [4]:
x = np.array(train_v3.iloc[:,1:42])
y = np.array(train_labels)

In [6]:
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(x, y):
    xgb_model = xgb.XGBRFRegressor(random_state=40).fit(
    x[train_index], y[train_index])

In [8]:
test_data = pd.read_csv('encoded_test_values.csv')

In [11]:
test_data = test_data.drop(labels=['land_surface_condition','foundation_type','roof_type',\
'ground_floor_type','other_floor_type','position', 'plan_configuration','legal_ownership_status',\
 'has_secondary_use','has_secondary_use_agriculture','has_secondary_use_hotel','has_secondary_use_rental',\
'has_secondary_use_institution','has_secondary_use_school','has_secondary_use_industry','has_secondary_use_health_post',\
'has_secondary_use_gov_office','has_secondary_use_use_police','has_secondary_use_other'],axis=1)

test_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86868 entries, 0 to 86867
Data columns (total 43 columns):
building_id                               86868 non-null int64
geo_level_1_id                            86868 non-null int64
geo_level_2_id                            86868 non-null int64
geo_level_3_id                            86868 non-null int64
count_floors_pre_eq                       86868 non-null int64
age                                       86868 non-null int64
area_percentage                           86868 non-null int64
height_percentage                         86868 non-null int64
has_superstructure_adobe_mud              86868 non-null int64
has_superstructure_mud_mortar_stone       86868 non-null int64
has_superstructure_stone_flag             86868 non-null int64
has_superstructure_cement_mortar_stone    86868 non-null int64
has_superstructure_mud_mortar_brick       86868 non-null int64
has_superstructure_cement_mortar_brick    86868 non-null int64
has_super

In [16]:
predictions = xgb_model.predict(np.array(test_data.iloc[:,1:42]))

In [19]:
from sklearn.metrics import accuracy_score

In [20]:
score = xgb_model.score(x, y)
print("accuracy=%.2f%%" % ( score * 100))

accuracy=31.94%


In [17]:
train = train_v3.drop(labels=[
 'has_superstructure_adobe_mud_mean','has_superstructure_mud_mortar_stone_mean','has_superstructure_stone_flag_mean','has_superstructure_cement_mortar_stone_mean',\
'has_superstructure_mud_mortar_brick_mean','has_superstructure_cement_mortar_brick_mean','has_superstructure_timber_mean','has_superstructure_bamboo_mean',\
'has_superstructure_rc_non_engineered_mean','has_superstructure_other_mean','has_superstructure_rc_engineered_mean'],axis=1)

In [18]:
test_data_v2 = test_data.drop(labels=[
 'has_superstructure_adobe_mud','has_superstructure_mud_mortar_stone','has_superstructure_stone_flag','has_superstructure_cement_mortar_stone',\
'has_superstructure_mud_mortar_brick','has_superstructure_cement_mortar_brick','has_superstructure_timber','has_superstructure_bamboo',\
'has_superstructure_rc_non_engineered','has_superstructure_other','has_superstructure_rc_engineered'],axis=1)

In [21]:
x2 = np.array(train.iloc[:,1:42])
y2 = np.array(train_labels)

In [23]:
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(x2, y2):
    xgb_model_v2 = xgb.XGBRFRegressor(random_state=40).fit(
    x2[train_index], y2[train_index])

In [24]:
score_v2 = xgb_model_v2.score(x2, y2)
print("accuracy=%.2f%%" % ( score_v2 * 100))

accuracy=31.60%


In [26]:
predictions_v2 = xgb_model_v2.predict(np.array(test_data_v2.iloc[:,1:42]))

In [27]:
##Tenemos 2 resultados con unas precision de acierto sobre el entrenamiento muy bajo

In [28]:
submit = pd.read_csv('submission_format.csv', index_col='building_id')
submission1= pd.DataFrame(
    data=predictions,
    columns=submit.columns,
    index=submit.index
)
submission1.head()

,damage_grade
building_id,
300051,2.060194
99355,1.998909
890251,1.900451
745817,1.564377
421793,2.044878


In [29]:
submit = pd.read_csv('submission_format.csv', index_col='building_id')
submission2= pd.DataFrame(
    data=predictions_v2,
    columns=submit.columns,
    index=submit.index
)
submission2.head()

,damage_grade
building_id,
300051,1.804363
99355,1.919075
890251,1.855331
745817,1.359991
421793,1.792161


In [31]:
submission2['damage_grade']=submission2['damage_grade'].apply(lambda x: round(x))
submission1['damage_grade']=submission1['damage_grade'].apply(lambda x: round(x))

In [32]:
submission2['damage_grade']=submission2['damage_grade'].astype('int64')
submission1['damage_grade']=submission1['damage_grade'].astype('int64')

In [34]:
submission2['damage_grade'].value_counts()

2    81359
1     4698
3      811
Name: damage_grade, dtype: int64

In [35]:
submission1['damage_grade'].value_counts()

2    81744
3     4995
1      129
Name: damage_grade, dtype: int64

In [36]:
submission2.to_csv('submitXGBRFRegressor2.csv') #No se subio a driven

In [37]:
submission1.to_csv('submitXGBRFRegressor1.csv') #Se tuvo un score de 0.56